#### CT-208 - Matemática Computacional


##### 1. Covariance Matrix Adaptation Evolution Strategy (CMA-ES)

- Modelo Evolutivo de 2ª Ordem;
- Modelos Evolutivos geram populações através de uma distribuição normal multivariada em $\mathbb{R}^n$, medem a performance de cada indivíduo, selecionam os p melhores para atualizar a média da distribuição para a próxima iteração.
- CMA atualiza não só a **média**, como também a **matriz de covariância** que gera a população.


##### Exemplo:
<img src="CMA.png" alt="fishy" class="bg-primary mb-1" width="500px" style={align:center}>


##### 2. Parsing SVG Files

In [1]:
import os
import re
import numpy as np

from PIL             import Image
from parseSvgFiles   import parseSVG
from convertSvg2Png  import convertSVG2PNG
from convertPng2Svg  import convertPNG2SVG
from placeComponent  import ComponentPlacing
from utils           import binarray2jpg, png2bin, selectComponentToPlace


Parsing each SVG file into separate components

In [2]:
# Select figure in folder and parse SVG file into separate components 
for svg_name in ['example', 'harry-potter', 'e190-e2']:
    parseSVG(svg_name)

Components from example separated with success!
Components from harry-potter separated with success!
Components from e190-e2 separated with success!


Convert each SVG component into a PNG equivalent

In [3]:
# Parse each SVG component into PNG equivalent 
for svg_name in ['example', 'harry-potter', 'e190-e2']:
    convertSVG2PNG(svg_name)

Components from example transformed with success!
Components from harry-potter transformed with success!
Components from e190-e2 transformed with success!


In [4]:
# Conversion parameters of each project
params = dict()
params['example'] = {'MM2PX': 1, 'PX2MM': 1}
params['harry-potter'] = {'MM2PX': 3.7795276032851057, 'PX2MM': 0.2645833302370423}
params['e190-e2'] = {'MM2PX': 1, 'PX2MM': 1}

# Sheet size
PAGE_SIZE = (210,297) 

In [5]:
# Define which project to optimize
#svg_name = 'example'
#svg_name = 'harry-potter'
svg_name = 'e190-e2'

# Conversion parameters
MM2PX = params[svg_name]['MM2PX']
PX2MM = params[svg_name]['PX2MM']

# Components and file paths
components_path = './' + svg_name + '/components_png'
test_results_path = './' + svg_name + '/test_results'

print("Conversion MM2PX: " + str(MM2PX))
print("Conversion PX2MM: " + str(PX2MM))
print("\nPaths:")
print(components_path)
print(test_results_path)

Conversion MM2PX: 1
Conversion PX2MM: 1

Paths:
./e190-e2/components_png
./e190-e2/test_results


In [6]:
# Finds the component png file in the directory
files = os.listdir(components_path)
pngs = list(filter(lambda name: re.match('.*\.png', name), files))

Examples of separated PNG components

In [7]:
from PIL import Image
with Image.open(components_path + "/" + np.random.choice(pngs)) as im:
    im.show()

Optimizer Parameters

In [8]:
# Optimizer parameters
iterations = 300
sigma = 1.0
population = 12

In [9]:
# Creates a list of pages and append a page of (height, lenght) px
pages = []
pages.append(np.zeros(PAGE_SIZE, dtype=int))
# Transforms each png component into a binary matrix
# List of binary matrix of components and its name
components = [[png2bin(components_path + '/' + png), png] for png in pngs] 
# Creates a list of objects of class ComponentPlacing
CP_list = [ComponentPlacing(pages[0], component, sigma=sigma, population=population, num_iterations=iterations) for component in components]


(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 3 (seed=565097, Wed Jun 29 00:10:24 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 3 (seed=501887, Wed Jun 29 00:10:24 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 3 (seed=564921, Wed Jun 29 00:10:24 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 3 (seed=578586, Wed Jun 29 00:10:24 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 3 (seed=503442, Wed Jun 29 00:10:24 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 3 (seed=551212, Wed Jun 29 00:10:24 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 3 (seed=594886, Wed Jun 29 00:10:24 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 3 (seed=575399, Wed Jun 29 00:10:24 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 3 (seed=578603, Wed Jun 29 00:10:24 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 3 (seed=506893, Wed Jun 29 00:10:24 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 3 (seed=566335, Wed Jun 29 00:10:24 2022)

In [10]:
# Place components on page
results = []
page_idx = 0
max_attempts = 100
num_components = len(CP_list)


In [11]:
while CP_list:
    attempt = 0     # counter of times that try to fit a component in a page
    # This loop tries to fit a component on current page    
    while attempt < max_attempts:
        # Selects one aleatory component
        idx = selectComponentToPlace(CP_list)
        CP = CP_list[idx]
        # Finds the best position
        CP.FindCandidatePosition()
        # Checks if there is no intersection and place the compoponent if not
        if CP.NoIntersection():
            pages[page_idx] = CP.PlaceComponentInPage()
            # removes the placed component from the list and store in results list
            results.append(CP_list.pop(idx)) 
            num_components -= 1
            print(('Current page: ' + str(page_idx+1)).ljust(18) + 'Components remaining: ' + str(num_components)+'...')
            
            if CP_list: 
                attempt = 0
            else: 
                break
        else: 
            attempt += 1
            CP.resetParameters() # sorts a new initial point
            
    # Prepares a new page if necessary
    if CP_list:
        pages.append(np.zeros(PAGE_SIZE, dtype=int))
        page_idx += 1
        for CP in CP_list:
            # sorts new initial point for the objects
            CP.page = pages[page_idx]
            CP.page_idx = page_idx
            CP.resetParameters()


Current page: 1   Components remaining: 54...
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 3 (seed=511050, Wed Jun 29 00:10:27 2022)
Current page: 1   Components remaining: 53...
Current page: 1   Components remaining: 52...
Current page: 1   Components remaining: 51...
Current page: 1   Components remaining: 50...
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 3 (seed=588326, Wed Jun 29 00:10:29 2022)
Current page: 1   Components remaining: 49...
Current page: 1   Components remaining: 48...
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 3 (seed=610213, Wed Jun 29 00:10:31 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 3 (seed=524171, Wed Jun 29 00:10:32 2022)
Current page: 1   Components remaining: 47...
Current page: 1   Components remaining: 46...
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 3 (seed=604240, Wed Jun 29 00:10:33 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 3 (seed=578151, Wed Jun 29 00:10:33 2022)
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in d

In [12]:
# Remove all png files from the folder before storing the results
files = os.listdir(test_results_path)
for file in files:
    os.remove(test_results_path + '/' + file)


In [13]:

# Saves pages
for idx,page in enumerate(pages):
    image = binarray2jpg(page).astype(np.uint8)
    image = Image.fromarray(image)
    image.save(test_results_path + '/page' + str(idx).zfill(2) + '.png')

print('Finished optimization.')

Finished optimization.


In [14]:
# Parse each PNG file into SVG file 
for svg_name in ['example', 'harry-potter', 'e190-e2']:
    convertPNG2SVG(svg_name)

ModuleNotFoundError: No module named 'place_component'